INSTALL DEPENDENCIES

In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install onnxmltools
!pip install onnxruntime
!pip install pyodbc
!pip install setuptools
!pip install skl2onnx
!pip install sqlalchemy

CREATE DATASET

In [66]:
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
import skl2onnx
import sklearn
import sklearn.datasets

from sklearn.datasets import load_boston
boston = load_boston()
boston

df = pd.DataFrame(data=np.c_[boston['data'], boston['target']], columns=boston['feature_names'].tolist() + ['MEDV'])
 
target_column = 'MEDV'
 
# Split the data frame into features and target
x_train = pd.DataFrame(df.drop([target_column], axis = 1))
y_train = pd.DataFrame(df.iloc[:,df.columns.tolist().index(target_column)])

print("\n*** Training dataset x\n")
print(x_train.head())

print("\n*** Training dataset y\n")
print(y_train.head())


*** Training dataset x

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  

*** Training dataset y

   MEDV
0  24.0
1  21.6
2  34.7
3  33.4
4  36.2


CREATE PIPELINE TO TRAIN THE LINEAR REGRESSION MODEL

In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])])),
                ('regressor', LinearRegression())])

CHECK MODEL ACCURACY

In [68]:
# Score the model
from sklearn.metrics import r2_score, mean_squared_error
y_pred = model.predict(x_train)
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.7406426641094094
*** Scikit-learn MSE: 21.894831181729206


DEFINE FUNCTION TO CONVERT MODEL TO ONNX

In [69]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

SAVE MODEL IN ONNX FORMAT

In [72]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(model, 'Boston Data', convert_dataframe_schema(x_train), final_types=[('variable1',FloatTensorType([1,1]))])
# Save the onnx model locally
onnx_model_path = 'boston1.model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)

TEST ONNX MODEL

In [74]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model_path)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = r2_score(y_train, y_pred)
onnx_mse = mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print('R2 Scores are equal' if sklearn_r2_score == onnx_r2_score else 'Difference in R2 scores: {}'.format(abs(sklearn_r2_score - onnx_r2_score)))
print('MSE are equal' if sklearn_mse == onnx_mse else 'Difference in MSE scores: {}'.format(abs(sklearn_mse - onnx_mse)))
print()


*** Onnx r2 score: 0.740642667298669
*** Onnx MSE: 21.894830912493322

Difference in R2 scores: 3.189259634162056e-09
Difference in MSE scores: 2.692358833655817e-07



<u>INSERT MODEL IN DATABASE</u>

In [86]:
import pyodbc

server = 'iotedgevm-be9601db-be9601db.eastus.cloudapp.azure.com' # SQL Server IP address
username = 'sa' # SQL Server username
password = 'SuperSecretP@ssw0rd!' # SQL Server password

# Connect to the master DB to create the new onnx database
connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=master;UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

database = 'onnx'
query = 'DROP DATABASE IF EXISTS ' + database
cursor.execute(query)
conn.commit()

# Create onnx database
query = 'CREATE DATABASE ' + database
cursor.execute(query)
conn.commit()

# Connect to onnx database
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()

models_table_name = 'models'

# Drop the table if it exists
query = f'drop table if exists {models_table_name}'
cursor.execute(query)
conn.commit()

# Create the model table
query = f'create table {models_table_name} ( ' \
    f'[id] [int] IDENTITY(1,1) NOT NULL, ' \
    f'[data] [varbinary](max) NULL, ' \
    f'[description] varchar(1000))'
cursor.execute(query)
conn.commit()

# Insert the ONNX model into the models table
query = f"insert into {models_table_name} ([description], [data]) values ('Onnx Model',?)"

model_bits = onnx_model.SerializeToString()

insert_params  = (pyodbc.Binary(model_bits))
cursor.execute(query, insert_params)
conn.commit()


ENSURE MODEL HAS BEEN STORED IN TABLE

In [92]:
conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()
query = f'SELECT * FROM {models_table_name}'
cursor.execute(query)

for row in cursor.fetchall():
    print(row)


(1, b'\x08\x07\x12\x08skl2onnx\x1a\x051.9.0"\x07ai.onnx(\x002\x00:\x9f\x0e\n)\n\x03NOX\x12\x08NOX_cast\x1a\x05Cast4"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n\'\n\x02ZN\x12\x07ZN_cast\x1a\x05Cast1"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n*\n\x04CRIM\x12\tCRIM_cast\x1a\x04Cast"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n-\n\x05INDUS\x12\nINDUS_cast\x1a\x05Cast2"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n\'\n\x02RM\x12\x07RM_cast\x1a\x05Cast5"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n&\n\x01B\x12\x06B_cast\x1a\x06Cast11"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n.\n\x05LSTAT\x12\nLSTAT_cast\x1a\x06Cast12"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n)\n\x03AGE\x12\x08AGE_cast\x1a\x05Cast6"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n)\n\x03DIS\x12\x08DIS_cast\x1a\x05Cast7"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n)\n\x03RAD\x12\x08RAD_cast\x1a\x05Cast8"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n)\n\x03TAX\x12\x08TAX_cast\x1a\x05Cast9"\x04Cast*\t\n\x02to\x18\x

CREATE FEATURES AND TARGET TABLES

In [93]:
import sqlalchemy
from sqlalchemy import create_engine
import urllib

db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(db_connection_string)
cursor = conn.cursor()

features_table_name = 'features'

# Drop the table if it exists
query = f'drop table if exists {features_table_name}'
cursor.execute(query)
conn.commit()

# Create the features table
query = \
    f'create table {features_table_name} ( ' \
    f'    [CRIM] float, ' \
    f'    [ZN] float, ' \
    f'    [INDUS] float, ' \
    f'    [CHAS] float, ' \
    f'    [NOX] float, ' \
    f'    [RM] float, ' \
    f'    [AGE] float, ' \
    f'    [DIS] float, ' \
    f'    [RAD] float, ' \
    f'    [TAX] float, ' \
    f'    [PTRATIO] float, ' \
    f'    [B] float, ' \
    f'    [LSTAT] float, ' \
    f'    [id] int)'

cursor.execute(query)
conn.commit()

target_table_name = 'target'

query = f'drop table if exists {target_table_name}'
cursor.execute(query)
conn.commit()

# Create the target table
query = \
    f'create table {target_table_name} ( ' \
    f'    [MEDV] float, ' \
    f'    [id] int)'

cursor.execute(query)
conn.commit()

x_train['id'] = range(1, len(x_train)+1)
y_train['id'] = range(1, len(y_train)+1)

print(x_train.head())
print(y_train.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  id  
0     15.3  396.90   4.98   1  
1     17.8  396.90   9.14   2  
2     17.8  392.83   4.03   3  
3     18.7  394.63   2.94   4  
4     18.7  396.90   5.33   5  
   MEDV  id
0  24.0   1
1  21.6   2
2  34.7   3
3  33.4   4
4  36.2   5


INSERT FEATURES & TARGET DATASETS INTO THE DATABASE

In [95]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

# for index, row in x_train.iterrows():
#     # print(index)
#     cursor.execute(f'INSERT INTO {features_table_name} (CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,id) VALUES ({row.CRIM},{row.ZN},{row.INDUS},{row.CHAS},{row.NOX},{row.RM},{row.AGE},{row.DIS},{row.RAD},{row.TAX},{row.PTRATIO},{row.B},{row.LSTAT},{row.id})')

# conn.commit()

for index, row in y_train.iterrows():
    # print(index)
    cursor.execute(f'INSERT INTO {target_table_name} (MEDV,id) VALUES ({row.MEDV},{row.id})')

conn.commit()
cursor.close()

In [ ]:
RUN PREDICT STATEMENT (SWITCH TO SQL KERNEL)

In [1]:
USE onnx

DECLARE @model VARBINARY(max) = (
        SELECT DATA
        FROM dbo.models
        WHERE id = 1
        );

WITH predict_input
AS (
    SELECT TOP (1000) [id]
        , CRIM
        , ZN
        , INDUS
        , CHAS
        , NOX
        , RM
        , AGE
        , DIS
        , RAD
        , TAX
        , PTRATIO
        , B
        , LSTAT
    FROM [dbo].[features]
    )
SELECT predict_input.id
    , p.variable1 AS MEDV
FROM PREDICT(MODEL = @model, DATA = predict_input, RUNTIME=ONNX) WITH (variable1 FLOAT) AS p;


Total execution time: 00:00:00.087

id,MEDV


: Msg 39051, Level 16, State 11, Line 9
Error occurred during execution of the builtin function 'PREDICT' with HRESULT 0x80004005. Model is corrupt or invalid.